In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
HOPSWORKS_PROJECT_NAME = 'bike_sharing_demand'

In [3]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [4]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])

File 2022 was already in local storage
File 2023 was already in local storage


In [5]:
print(f'{len(rides)=}')

len(rides)=4829258


In [6]:
rides.pickup_datetime.max()

Timestamp('2023-09-30 23:58:56')

In [7]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 362/362 [00:09<00:00, 39.72it/s]


In [8]:
ts_data.tail(730)

,pickup_hour,rides,pickup_location_id
5067542,2023-08-31 14:00:00,1,57
5067543,2023-08-31 15:00:00,3,57
5067544,2023-08-31 16:00:00,1,57
5067545,2023-08-31 17:00:00,1,57
5067546,2023-08-31 18:00:00,0,57
...,...,...,...
5068267,2023-09-30 19:00:00,0,57
5068268,2023-09-30 20:00:00,0,57
5068269,2023-09-30 21:00:00,0,57
5068270,2023-09-30 22:00:00,3,57


In [9]:
import hopsworks

print(hopsworks.__version__)

3.4.2


In [10]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/100501


In [11]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [12]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [13]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [14]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/100501/fs/100420/fg/280937


Uploading Dataframe: 0.00% |          | Rows 0/5068272 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/100501/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2100bd63280>, None)